<a id='imports_notebook'></a>
### Imports del notebook
[Volver al índice](#index)

En este apartado hacemos los principales imports del notebook.<br>
Sobre todo vamos a trabajar con **numpy**, **pandas**, **statsmodels** y **XGBoost**.

In [ ]:
pip install statsmodels

In [ ]:
pip install xgboost

In [ ]:
import os

import pandas as pd
import xgboost as xgb

In [ ]:
print(f"pandas version: {pd.__version__}")
print(f"xgboost version: {xgb.__version__}")

pandas version: 2.1.4
xgboost version: 2.1.1


In [ ]:
import pickle
import os

from datetime import datetime
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder

import xgboost as xgb

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [ ]:
print("Working with these versions of libraries\n")
print("-"*50)
print(f"Numpy version {np.__version__}")
print(f"Pandas version {pd.__version__}")
print(f"XGBoost version {xgb.__version__}")

Working with these versions of libraries

--------------------------------------------------
Numpy version 1.26.4
Pandas version 2.1.4
XGBoost version 2.1.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
CWD = os.getcwd()
DATA_PATH = '/content/drive/MyDrive/Master_Data_Science/TFM/Forecasting'

FILE_NAME = "df_DSMARKET_FORECASTING" # Nombre del pickle de partida para el experimento
DATAFRAME_PATH = os.path.join(DATA_PATH, FILE_NAME)

### Data Loading

Cargo el dataset con la columna yearweek modificada para que la semana empiece el lunes y con los outliers de price_per_unit y units_sold reemplazados.

In [ ]:
df = pd.read_pickle(DATAFRAME_PATH)

In [ ]:
pd.set_option('display.max_columns',None)

In [ ]:
df

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag,week_of_month,best_week_of_the_month,best_month_of_quarter,best_quarter_of_year
0,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-29,2011,1,1,201104,Saturday,6,29,1,0.0,10.968041,0.0,No_event,4,0,False,5,False,False,False
1,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-30,2011,1,1,201104,Sunday,7,30,2,0.0,10.968041,0.0,No_event,4,0,False,5,False,False,False
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105,Monday,1,31,3,0.0,10.968041,0.0,No_event,4,0,True,6,False,False,False
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105,Tuesday,2,1,4,0.0,10.968041,0.0,No_event,4,0,True,1,False,False,False
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105,Wednesday,3,2,5,0.0,10.968041,0.0,No_event,4,0,True,1,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58289105,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-20,2016,2,4,201616,Wednesday,3,20,1909,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False
58289106,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-21,2016,2,4,201616,Thursday,4,21,1910,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False
58289107,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-22,2016,2,4,201616,Friday,5,22,1911,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False
58289108,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-23,2016,2,4,201616,Saturday,6,23,1912,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False


Elimino el primer yearweek (la primera semana porque incompleta)

In [ ]:
df = df[df['yearweek']!=201104]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 58228170 entries, 2 to 58289109
Data columns (total 27 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   id                      object        
 1   item                    object        
 2   category                object        
 3   department              object        
 4   store                   object        
 5   store_code              object        
 6   region                  object        
 7   date                    datetime64[ns]
 8   year                    int64         
 9   quarter                 int32         
 10  month                   int64         
 11  yearweek                int64         
 12  weekday                 object        
 13  weekday_int             int32         
 14  day                     int64         
 15  d                       int64         
 16  units_sold              float64       
 17  price_per_unit          float64       
 18  reven

In [ ]:
df['date'].dt.isocalendar().week

,week
2,5
3,5
4,5
5,5
6,5
...,...
58289105,16
58289106,16
58289107,16
58289108,16


In [ ]:
df.insert(12,'week_of_year',df['date'].dt.isocalendar().week)

In [ ]:
df

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag,week_of_month,best_week_of_the_month,best_month_of_quarter,best_quarter_of_year
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105,5,Monday,1,31,3,0.0,10.968041,0.0,No_event,4,0,True,6,False,False,False
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105,5,Tuesday,2,1,4,0.0,10.968041,0.0,No_event,4,0,True,1,False,False,False
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105,5,Wednesday,3,2,5,0.0,10.968041,0.0,No_event,4,0,True,1,False,False,False
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011,1,2,201105,5,Thursday,4,3,6,0.0,10.968041,0.0,No_event,4,0,True,1,False,False,False
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011,1,2,201105,5,Friday,5,4,7,0.0,10.968041,0.0,No_event,4,0,True,1,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58289105,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-20,2016,2,4,201616,16,Wednesday,3,20,1909,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False
58289106,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-21,2016,2,4,201616,16,Thursday,4,21,1910,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False
58289107,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-22,2016,2,4,201616,16,Friday,5,22,1911,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False
58289108,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-23,2016,2,4,201616,16,Saturday,6,23,1912,0.0,1.200000,0.0,No_event,3,0,False,4,False,True,False


In [ ]:
df.drop(['best_quarter_of_year','best_month_of_quarter','best_week_of_the_month'],axis=1,inplace=True)

<ipython-input-19-28279f8bedeb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['best_quarter_of_year','best_month_of_quarter','best_week_of_the_month'],axis=1,inplace=True)


In [ ]:
df

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag,week_of_month
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105,5,Monday,1,31,3,0.0,10.968041,0.0,No_event,4,0,True,6
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105,5,Tuesday,2,1,4,0.0,10.968041,0.0,No_event,4,0,True,1
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105,5,Wednesday,3,2,5,0.0,10.968041,0.0,No_event,4,0,True,1
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011,1,2,201105,5,Thursday,4,3,6,0.0,10.968041,0.0,No_event,4,0,True,1
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011,1,2,201105,5,Friday,5,4,7,0.0,10.968041,0.0,No_event,4,0,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58289105,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-20,2016,2,4,201616,16,Wednesday,3,20,1909,0.0,1.200000,0.0,No_event,3,0,False,4
58289106,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-21,2016,2,4,201616,16,Thursday,4,21,1910,0.0,1.200000,0.0,No_event,3,0,False,4
58289107,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-22,2016,2,4,201616,16,Friday,5,22,1911,0.0,1.200000,0.0,No_event,3,0,False,4
58289108,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-04-23,2016,2,4,201616,16,Saturday,6,23,1912,0.0,1.200000,0.0,No_event,3,0,False,4


In [ ]:
df.columns

Index(['id', 'item', 'category', 'department', 'store', 'store_code', 'region',
       'date', 'year', 'quarter', 'month', 'yearweek', 'week_of_year',
       'weekday', 'weekday_int', 'day', 'd', 'units_sold', 'price_per_unit',
       'revenue', 'event', 'product_cluster', 'store_cluster',
       'Event_week_flag', 'week_of_month'],
      dtype='object')

# Añadir los 28 dias futuros (in process)

Añado los 28 dias requeridos para la prediccion futura

In [ ]:
# Función para agregar 28 días a cada grupo
def add_28_days(group):
    # Extraer la última fecha del grupo
    last_date = group['date'].max()

    # Crear nuevas fechas
    new_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=28)

    # Crear un nuevo DataFrame con estas nuevas fechas y valores NaN para otras columnas
    new_rows = pd.DataFrame({
        'id': group['id'].iloc[0],
        'item': group['item'].iloc[0],
        'category': group['category'].iloc[0],
        'department': group['department'].iloc[0],
        'store': group['store'].iloc[0],
        'store_code': group['store_code'].iloc[0],
        'region': group['region'].iloc[0],
        'product_cluster': group['product_cluster'].iloc[0],
        'store_cluster': group['store_cluster'].iloc[0],
        'date': new_dates,
        'units_sold': np.nan,
        'price_per_unit': np.nan,
        'revenue': np.nan,
        'Event_week_flag': np.nan,
    })

    # Concatenar el grupo original con las nuevas filas
    return pd.concat([group, new_rows])


In [ ]:
# Aplicar la función a cada grupo de series temporales
df_extended = df.groupby('id', group_keys=False).apply(add_28_days)

In [ ]:
# Mostrar el DataFrame resultante
df_extended

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag,week_of_month
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011.0,1.0,1.0,201105.0,5,Monday,1.0,31.0,3.0,0.0,10.968041,0.0,No_event,4,0,1.0,6.0
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011.0,1.0,2.0,201105.0,5,Tuesday,2.0,1.0,4.0,0.0,10.968041,0.0,No_event,4,0,1.0,1.0
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011.0,1.0,2.0,201105.0,5,Wednesday,3.0,2.0,5.0,0.0,10.968041,0.0,No_event,4,0,1.0,1.0
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011.0,1.0,2.0,201105.0,5,Thursday,4.0,3.0,6.0,0.0,10.968041,0.0,No_event,4,0,1.0,1.0
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011.0,1.0,2.0,201105.0,5,Friday,5.0,4.0,7.0,0.0,10.968041,0.0,No_event,4,0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-18,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,NaN,NaN
24,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-19,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,NaN,NaN
25,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-20,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,NaN,NaN
26,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-21,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0,NaN,NaN


In [ ]:
df['product_cluster'].unique()

array([4, 5, 1, 3, 2])

In [ ]:
df['store_cluster'].unique()

array([0, 3, 1, 2])

In [ ]:
df['store_cluster'] = df['store_cluster'].replace(0,1)

<ipython-input-27-24c467834733>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['store_cluster'] = df['store_cluster'].replace(0,1)


Trato los nulos

In [ ]:
df_extended['year'] = df_extended['date'].dt.year
df_extended['quarter'] = df_extended['date'].dt.quarter
df_extended['month'] = df_extended['date'].dt.month
df_extended['week_of_year'] = df_extended['date'].dt.isocalendar().week
df_extended['weekday'] = df_extended['date'].dt.day_name()
df_extended['day'] = df_extended['date'].dt.day

Trato los nulos en week of month

In [ ]:
# Extraer el día del mes y el día de la semana del primer día del mes
df_extended['day_of_month'] = df_extended['date'].dt.day
df_extended['first_day_of_month'] = df_extended['date'].dt.to_period('M').dt.start_time
df_extended['day_of_week_first'] = df_extended['first_day_of_month'].dt.dayofweek

# Calcular la semana del mes
df_extended['week_of_month'] = ((df_extended['day_of_month'] + df_extended['day_of_week_first'] - 1) // 7) + 1

In [ ]:
df_extended

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag,week_of_month,day_of_month,first_day_of_month,day_of_week_first
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105.0,5,Monday,1.0,31,3.0,0.0,10.968041,0.0,No_event,4,0,1.0,6,31,2011-01-01,5
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105.0,5,Tuesday,2.0,1,4.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,1,2011-02-01,1
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105.0,5,Wednesday,3.0,2,5.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,2,2011-02-01,1
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011,1,2,201105.0,5,Thursday,4.0,3,6.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,3,2011-02-01,1
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011,1,2,201105.0,5,Friday,5.0,4,7.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,4,2011-02-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-18,2016,2,5,NaN,20,Wednesday,NaN,18,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,18,2016-05-01,6
24,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-19,2016,2,5,NaN,20,Thursday,NaN,19,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,19,2016-05-01,6
25,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-20,2016,2,5,NaN,20,Friday,NaN,20,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,20,2016-05-01,6
26,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-21,2016,2,5,NaN,20,Saturday,NaN,21,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,21,2016-05-01,6


Trato los nulos en yearweek

In [ ]:
def calculate_yearweek(date):
    yearweek = date.isocalendar()
    year = yearweek.year
    week = yearweek.week
    return f"{year}{str(week).zfill(2)}"

# Aplicar la función a la columna 'date' y crear la columna 'yearweek'
df_extended['yearweek'] = df_extended['date'].apply(calculate_yearweek)

In [ ]:
df_extended['yearweek'] = df_extended['yearweek'].astype(int)

In [ ]:
df_extended

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag,week_of_month,day_of_month,first_day_of_month,day_of_week_first
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105,5,Monday,1.0,31,3.0,0.0,10.968041,0.0,No_event,4,0,1.0,6,31,2011-01-01,5
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105,5,Tuesday,2.0,1,4.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,1,2011-02-01,1
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105,5,Wednesday,3.0,2,5.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,2,2011-02-01,1
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011,1,2,201105,5,Thursday,4.0,3,6.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,3,2011-02-01,1
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011,1,2,201105,5,Friday,5.0,4,7.0,0.0,10.968041,0.0,No_event,4,0,1.0,1,4,2011-02-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-18,2016,2,5,201620,20,Wednesday,NaN,18,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,18,2016-05-01,6
24,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-19,2016,2,5,201620,20,Thursday,NaN,19,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,19,2016-05-01,6
25,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-20,2016,2,5,201620,20,Friday,NaN,20,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,20,2016-05-01,6
26,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-21,2016,2,5,201620,20,Saturday,NaN,21,NaN,NaN,NaN,NaN,NaN,3,0,NaN,4,21,2016-05-01,6


# Drop week_of_month

In [ ]:
df_extended.drop('week_of_month',axis=1,inplace=True)

In [ ]:
df_extended.drop(['day_of_month','first_day_of_month','day_of_week_first'],axis=1,inplace=True)

In [ ]:
df_extended

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105,5,Monday,1.0,31,3.0,0.0,10.968041,0.0,No_event,4,0,1.0
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105,5,Tuesday,2.0,1,4.0,0.0,10.968041,0.0,No_event,4,0,1.0
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105,5,Wednesday,3.0,2,5.0,0.0,10.968041,0.0,No_event,4,0,1.0
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011,1,2,201105,5,Thursday,4.0,3,6.0,0.0,10.968041,0.0,No_event,4,0,1.0
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011,1,2,201105,5,Friday,5.0,4,7.0,0.0,10.968041,0.0,No_event,4,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-18,2016,2,5,201620,20,Wednesday,NaN,18,NaN,NaN,NaN,NaN,NaN,3,0,NaN
24,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-19,2016,2,5,201620,20,Thursday,NaN,19,NaN,NaN,NaN,NaN,NaN,3,0,NaN
25,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-20,2016,2,5,201620,20,Friday,NaN,20,NaN,NaN,NaN,NaN,NaN,3,0,NaN
26,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-21,2016,2,5,201620,20,Saturday,NaN,21,NaN,NaN,NaN,NaN,NaN,3,0,NaN


In [ ]:
df_extended[df_extended['yearweek'].isin(range(201617,201621))]['date'].describe()

,date
count,853160
mean,2016-05-08 12:00:00
min,2016-04-25 00:00:00
25%,2016-05-01 18:00:00
50%,2016-05-08 12:00:00
75%,2016-05-15 06:00:00
max,2016-05-22 00:00:00


Trato los nulos en price_per_unit

In [ ]:
df_extended['price_per_unit'] = df_extended.groupby('id')['price_per_unit'].ffill()

In [ ]:
df_extended.isnull().sum()

,0
id,0
item,0
category,0
department,0
store,0
store_code,0
region,0
date,0
year,0
quarter,0


actualizo la columna revenue

In [ ]:
df_extended['revenue'] = df_extended['units_sold'] * df_extended['price_per_unit']

Trato los nulos en event

In [ ]:
df_extended[(df_extended['event'].notnull())&(df_extended['event']!='No_event')&(df_extended['month'].isin([4,5]))][['event','date']]

,event,date
435,Easter,2012-04-08
1177,Easter,2014-04-20
1527,Easter,2015-04-05
2348,Easter,2012-04-08
3090,Easter,2014-04-20
...,...,...
58286461,Easter,2014-04-20
58286811,Easter,2015-04-05
58287632,Easter,2012-04-08
58288374,Easter,2014-04-20


In [ ]:
df_extended[(df_extended['event'].notnull())&(df_extended['event']!='No_event')&(df_extended['month'].isin([4,5]))]['event'].unique()

array(['Easter'], dtype=object)

En el periodo que tenemos que predecir no hay eventos, asi que sustituire los nulos con No_event

In [ ]:
df_extended['event'] = df_extended['event'].fillna('No_event')

Trato los nulos en event_week_flag

In [ ]:
df_extended['Event_week_flag'] = df_extended['Event_week_flag'].fillna(False)

In [ ]:
df_extended['yearweek'].describe()

,yearweek
count,5.908133e+07
mean,2.013511e+05
std,1.538424e+02
min,2.011050e+05
25%,2.012220e+05
50%,2.013390e+05
75%,2.015040e+05
max,2.016200e+05


In [ ]:
df_extended

,id,item,category,department,store,store_code,region,date,year,quarter,month,yearweek,week_of_year,weekday,weekday_int,day,d,units_sold,price_per_unit,revenue,event,product_cluster,store_cluster,Event_week_flag
2,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-01-31,2011,1,1,201105,5,Monday,1.0,31,3.0,0.0,10.968041,0.0,No_event,4,0,1
3,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-01,2011,1,2,201105,5,Tuesday,2.0,1,4.0,0.0,10.968041,0.0,No_event,4,0,1
4,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-02,2011,1,2,201105,5,Wednesday,3.0,2,5.0,0.0,10.968041,0.0,No_event,4,0,1
5,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-03,2011,1,2,201105,5,Thursday,4.0,3,6.0,0.0,10.968041,0.0,No_event,4,0,1
6,ACCESORIES_1_001_BOS_1,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,South_End,BOS_1,Boston,2011-02-04,2011,1,2,201105,5,Friday,5.0,4,7.0,0.0,10.968041,0.0,No_event,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-18,2016,2,5,201620,20,Wednesday,NaN,18,NaN,NaN,1.200000,NaN,No_event,3,0,0
24,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-19,2016,2,5,201620,20,Thursday,NaN,19,NaN,NaN,1.200000,NaN,No_event,3,0,0
25,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-20,2016,2,5,201620,20,Friday,NaN,20,NaN,NaN,1.200000,NaN,No_event,3,0,0
26,SUPERMARKET_3_827_PHI_3,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,Queen_Village,PHI_3,Philadelphia,2016-05-21,2016,2,5,201620,20,Saturday,NaN,21,NaN,NaN,1.200000,NaN,No_event,3,0,0


In [ ]:
df_extended['Event_week_flag'] = df_extended['Event_week_flag'].astype(int)

In [ ]:
# Agrupar por 'id' y 'yearweek'
weekly_df = df_extended.groupby(['id', 'yearweek']).agg({
    'date':'last',
    'item': 'first',
    'category': 'first',
    'department': 'first',
    'store_code': 'first',
    'store': 'first',
    'region': 'first',
    'units_sold': 'sum',                            # Sum of the units sold per week
    'revenue': 'sum',                               # Sum of the revenue per week
    'price_per_unit': 'mean',                       # Average price per unit for the week
    'Event_week_flag': 'max',                       # True if any day in the week had an event
    'product_cluster': 'first',                     # Keep the product cluster for each ID
    'store_cluster': 'first',                       # Keep the store cluster for each ID
    'week_of_year': 'first',
    'month': 'first',                               # Keep the month for each ID
    'quarter': 'first',                             # Keep the quarter for each ID
    'year': 'first'                                 # Keep the year for each ID

}).reset_index()


weekly_df = weekly_df.sort_values(by=['id', 'yearweek'])

In [ ]:
weekly_df

,id,yearweek,date,item,category,department,store_code,store,region,units_sold,revenue,price_per_unit,Event_week_flag,product_cluster,store_cluster,week_of_year,month,quarter,year
0,ACCESORIES_1_001_BOS_1,201105,2011-02-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,1,4,0,5,1,1,2011
1,ACCESORIES_1_001_BOS_1,201106,2011-02-13,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,0,6,2,1,2011
2,ACCESORIES_1_001_BOS_1,201107,2011-02-20,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,0,7,2,1,2011
3,ACCESORIES_1_001_BOS_1,201108,2011-02-27,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,0,8,2,1,2011
4,ACCESORIES_1_001_BOS_1,201109,2011-03-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,0,9,2,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8440185,SUPERMARKET_3_827_PHI_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,0,16,4,2,2016
8440186,SUPERMARKET_3_827_PHI_3,201617,2016-05-01,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,0,17,4,2,2016
8440187,SUPERMARKET_3_827_PHI_3,201618,2016-05-08,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,0,18,5,2,2016
8440188,SUPERMARKET_3_827_PHI_3,201619,2016-05-15,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,0,19,5,2,2016


In [ ]:
weekly_df.rename(columns={'Event_week_flag':'event_week_flag'},inplace=True)

In [ ]:
weekly_df['store_cluster'] = weekly_df['store_cluster'].replace(0,1)

In [ ]:
weekly_df

,id,yearweek,date,item,category,department,store_code,store,region,units_sold,revenue,price_per_unit,event_week_flag,product_cluster,store_cluster,week_of_year,month,quarter,year
0,ACCESORIES_1_001_BOS_1,201105,2011-02-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,1,4,1,5,1,1,2011
1,ACCESORIES_1_001_BOS_1,201106,2011-02-13,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,6,2,1,2011
2,ACCESORIES_1_001_BOS_1,201107,2011-02-20,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,7,2,1,2011
3,ACCESORIES_1_001_BOS_1,201108,2011-02-27,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,8,2,1,2011
4,ACCESORIES_1_001_BOS_1,201109,2011-03-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,9,2,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8440185,SUPERMARKET_3_827_PHI_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,16,4,2,2016
8440186,SUPERMARKET_3_827_PHI_3,201617,2016-05-01,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,17,4,2,2016
8440187,SUPERMARKET_3_827_PHI_3,201618,2016-05-08,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,18,5,2,2016
8440188,SUPERMARKET_3_827_PHI_3,201619,2016-05-15,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,19,5,2,2016


In [ ]:
weekly_df['product_cluster'].unique()

array([4, 5, 1, 3, 2])

In [ ]:
weekly_df['store_cluster'].unique()

array([1, 3, 2])

### Exportación pickle sin encoding

In [ ]:
weekly_df.to_pickle('/content/drive/MyDrive/TFM/Forecasting/weekly_df_extended')

In [ ]:
weekly_df = pd.read_pickle('/content/drive/MyDrive/TFM/Forecasting/weekly_df_extended')

## Encoding object columns

In [ ]:
weekly_df

,id,yearweek,date,item,category,department,store_code,store,region,units_sold,revenue,price_per_unit,event_week_flag,product_cluster,store_cluster,week_of_year,month,quarter,year
0,ACCESORIES_1_001_BOS_1,201105,2011-02-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,1,4,1,5,1,1,2011
1,ACCESORIES_1_001_BOS_1,201106,2011-02-13,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,6,2,1,2011
2,ACCESORIES_1_001_BOS_1,201107,2011-02-20,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,7,2,1,2011
3,ACCESORIES_1_001_BOS_1,201108,2011-02-27,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,8,2,1,2011
4,ACCESORIES_1_001_BOS_1,201109,2011-03-06,ACCESORIES_1_001,ACCESORIES,ACCESORIES_1,BOS_1,South_End,Boston,0.0,0.0,10.968041,0,4,1,9,2,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8440185,SUPERMARKET_3_827_PHI_3,201616,2016-04-24,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,16,4,2,2016
8440186,SUPERMARKET_3_827_PHI_3,201617,2016-05-01,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,17,4,2,2016
8440187,SUPERMARKET_3_827_PHI_3,201618,2016-05-08,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,18,5,2,2016
8440188,SUPERMARKET_3_827_PHI_3,201619,2016-05-15,SUPERMARKET_3_827,SUPERMARKET,SUPERMARKET_3,PHI_3,Queen_Village,Philadelphia,0.0,0.0,1.200000,0,3,1,19,5,2,2016


In [ ]:
weekly_df['store_code'].unique()

array(['BOS_1', 'BOS_2', 'BOS_3', 'NYC_1', 'NYC_2', 'NYC_3', 'NYC_4',
       'PHI_1', 'PHI_2', 'PHI_3'], dtype=object)

In [ ]:
for code,num in zip(weekly_df['store_code'].unique(),range(len(weekly_df['store_code'].unique()))):
  print(f'{code}={num}')

BOS_1=0
BOS_2=1
BOS_3=2
NYC_1=3
NYC_2=4
NYC_3=5
NYC_4=6
PHI_1=7
PHI_2=8
PHI_3=9


In [ ]:
for code,num in zip(weekly_df['category'].unique(),range(len(weekly_df['category'].unique()))):
  print(f'{code}={num}')

ACCESORIES=0
HOME_&_GARDEN=1
SUPERMARKET=2


In [ ]:
for code,num in zip(weekly_df['department'].unique(),range(len(weekly_df['department'].unique()))):
  print(f'{code}={num}')

ACCESORIES_1=0
ACCESORIES_2=1
HOME_&_GARDEN_1=2
HOME_&_GARDEN_2=3
SUPERMARKET_1=4
SUPERMARKET_2=5
SUPERMARKET_3=6


In [ ]:
weekly_df_encoded = weekly_df.copy()

In [ ]:
# Inicializa OrdinalEncoder
encoder = OrdinalEncoder()

# Lista de columnas a codificar
columns_to_encode = ['item', 'category', 'department', 'store_code', 'region']

# Aplica OrdinalEncoder a cada columna individualmente
for col in columns_to_encode:
    weekly_df_encoded[col] = encoder.fit_transform(weekly_df_encoded[[col]])


In [ ]:
weekly_df_encoded

,id,yearweek,item,category,department,store_code,region,units_sold,revenue,price_per_unit,event_week_flag,product_cluster,store_cluster,week_of_year,month,quarter,year
0,ACCESORIES_1_001_BOS_1,201105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.968041,1,4,1,5,1,1,2011
1,ACCESORIES_1_001_BOS_1,201106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.968041,0,4,1,6,2,1,2011
2,ACCESORIES_1_001_BOS_1,201107,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.968041,0,4,1,7,2,1,2011
3,ACCESORIES_1_001_BOS_1,201108,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.968041,0,4,1,8,2,1,2011
4,ACCESORIES_1_001_BOS_1,201109,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.968041,0,4,1,9,2,1,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8440185,SUPERMARKET_3_827_PHI_3,201616,3046.0,2.0,6.0,9.0,2.0,0.0,0.0,1.200000,0,3,1,16,4,2,2016
8440186,SUPERMARKET_3_827_PHI_3,201617,3046.0,2.0,6.0,9.0,2.0,0.0,0.0,1.200000,0,3,1,17,4,2,2016
8440187,SUPERMARKET_3_827_PHI_3,201618,3046.0,2.0,6.0,9.0,2.0,0.0,0.0,1.200000,0,3,1,18,5,2,2016
8440188,SUPERMARKET_3_827_PHI_3,201619,3046.0,2.0,6.0,9.0,2.0,0.0,0.0,1.200000,0,3,1,19,5,2,2016


In [ ]:
weekly_df_encoded.drop(columns=['store','date'],inplace=True)

### Exportación pickle con encoding

In [ ]:
weekly_df_encoded.to_pickle('/content/drive/MyDrive/TFM/Forecasting/weekly_df_extended_encoded')

In [ ]:
weekly_df_encoded = pd.read_pickle('/content/drive/MyDrive/TFM/Forecasting/weekly_df_extended_encoded')